In [ ]:
import os
import shutil
import pandas as pd
import numpy as np

In [ ]:
# rename all the excels to year-index and copy to a new folder

In [ ]:
basedir = "/Users/sun/Desktop/Econ580Data/"
movedir = basedir + "all/"

# clean all the files in all first 
if os.path.isdir(movedir):
    shutil.rmtree(movedir)
os.mkdir(movedir)

# used to grant permission
# permission_array = []
# Walk through all files in the directory that contains the files to copy to the "all" folder
for root, dirs, files in os.walk(basedir):
    # use data from 1999 to 2012
    if not root[-4:].isdigit():
        continue
    if int(root[-4:]) < 2000 or int(root[-4:]) > 2012:
        continue
    
    index = 1
    for file in files:
        ori_file = os.path.abspath(root) + "/" + file
        base, extension = os.path.splitext(file)

        if extension != ".xls":
            continue
        # file year-index.xls
        new_file = movedir + root[-4:] + "-" + str(index) + extension
        
        index += 1
        # break
        shutil.copy(ori_file, new_file)
        # permission_array.append(new_file)

print("Done")
# print("\"" + "\",  _\n\"""".join(permission_array) + "\"" )

In [ ]:
# xls2xlsx, in vba file
# references: 
# 1. translate xls to xlsx: https://answers.microsoft.com/en-us/msoffice/forum/msoffice_excel-mso_other/batch-convert-xls-to-xlsx/1d9b3d78-daf0-4014-8fb2-930aca6493b0
# 2. request permission for all: https://docs.microsoft.com/en-us/office/vba/office-mac/grantaccesstomultiplefiles


In [ ]:
# remove the xls file if it is already converted to xlsx
basedir = "/Users/sun/Desktop/Econ580Data/"
movedir = basedir + "all/"

# Walk through all files in the directory that contains the files to copy to the "all" folder
for file in os.listdir(movedir):
    # should only have xls and xlsx
    ori_file = movedir + file
    base, extension = os.path.splitext(file)

    if extension != ".xlsx":
        continue
    # is now filename.xlsx
    # remoove the xls based on xlsx file
    # print(movedir + file[:-1])
    if os.path.exists(movedir + file[:-1]):
        # print("exist")
        os.remove(movedir + file[:-1]) 
    else:
        # print("xls not exist: ", movedir + file[:-1])
        pass
    
# check whether there are xls file not converted to xlsx
xls_file = [f for f in os.listdir(movedir) if os.path.splitext(f)[1] == ".xls"]
print(xls_file)

In [ ]:
# Format
basedir = "/Users/sun/Desktop/Econ580Data/"
movedir = basedir + "all/"
outdir = basedir + "formated/"

# 2000
file_path = movedir+"2000-2.xlsx"
xls = pd.ExcelFile(file_path)
# out_path = outdir + "2000.csv"

for year in range(2012,2013):
    file_province = {} # used to check xlsx file if format error
    all_data = []
    out_path = outdir + str(year) + ".csv"
    for cur_file in os.listdir(movedir):
        cur_provinces = [] # add to province dic

        # should only have xls and xlsx
        ori_file = movedir + cur_file
        base, extension = os.path.splitext(cur_file)

        if extension != ".xlsx":
            continue
        
        # if "2012-16.xlsx" not in cur_file or "~$" in cur_file:
        if str(year) not in cur_file or "~$" in cur_file:
            continue

        # open file and parse
        print(cur_file)

        file_path = movedir+ cur_file
        xls = pd.ExcelFile(file_path)           
        # Iterate through worksheets in opened Excel file
        for sheet in xls.sheet_names:
            # Create a Pandas dataframe from the Excel worksheet (with no headers)
            dfs = pd.read_excel(file_path, sheet_name=sheet, header=None)
            if year == 2012:
                # worksheet for year 2012 are vertically appended, others are appended horizontally
                list_index = dfs[dfs.isnull().all(1)].index.tolist()
                df_list = np.split(dfs, list_index, axis = 0)
                # remove all null column
                df_list = [df_list[x] for x in range(len(df_list)) if not df_list[x].isnull().all().all()]               
            else:
                list_index = [x for x in range(dfs.shape[1]) if dfs[:][x].isnull().all(0) ]
                # separated by 2 null column
                df_list = np.split(dfs, list_index, axis = 1)
                # remove all null column
                df_list = [df_list[x] for x in range(len(df_list)) if not df_list[x].isnull().all().all()]
                # print(len(df_list))
            
            for df in df_list:
                # remove null column and row
                df = df.dropna(how="all", axis=0).dropna(how="all", axis=1).values.tolist()
                # filter out wrong df
                if len(df) < 30:
                    print("wrong sheet:", df)
                    continue
                # strip all the elements
                df = [[str(x).strip() if x!= np.nan else "" for x in y] for y in df]
                # first line would be year e.g. 2000年县(市)社会经济主要指标
                
                if "经济主要" in df[0][0]:
                    # check province whether in the first line
                    if df[0][0].strip()[0].isdigit():
                        df = df[1:]
                        province = df[0][0]
                    else:
                        # province in this line
                        # e.g 新疆维吾尔自治区 2006年县(市)社会经济主要指标统计
                        privince = df[0][0].strip().split(" ")[0]

                # province in [0][0], get province, remove first line
                # get county, start in second line, might change line in between
                #  e.g.['    指标' '  单位' '    桐庐县' '    淳安县']
                # print(df)
                county_end = [x for x in range(len(df)) if "一、" in df[x][0]]
                if county_end != []:
                    cur = df[1]
                    for i in range(2, county_end[0]):
                        cur = [ (cur[x] + df[i][x]).replace("nan", "").replace("\n","") for x in range(len(cur)) ]
                    counties = cur[2:]
                    df = df[county_end[0]:]
                    df = np.delete(df, 1, 1).tolist()
                else:
                    # this case: split between the worksheet
                    # e.g. 2009-17, sheet 9, 13, 16, no index name in the left
                    county_end = [x for x in range(len(df)) if df[x][0].strip().isdigit()][0]
                    counties = df[0]
                    for i in range(2, county_end):
                        counties = [ (counties[x] + df[i][x]).replace("nan", "").replace("\n","") for x in range(len(cur)) ]
                    df = df[county_end:]

                counties = [x.replace("nan", "").replace("\n","").replace("\r","") for x in counties]    
                # print(province, counties)
                # unit in second column, remove column
                df = [y for y in df if ( ("、" not in "".join(y) and "，" not in "".join(y)) or "医院、卫生院床位数" in "".join(y)) and y[0] != "" ]
                # df = pd.DataFrame(data=df).replace("nan", np.nan).dropna(how="any", axis=0).values.tolist()
                # add province and to the front
                df.insert(0, ["县"] + counties)
                df.insert(0, ["省"] + [province for x in range(len(counties))] )
                df.insert(0, ["年"] + [year for x in range(len(counties))])
                # transpose
                df = np.transpose(df).tolist()
                # print(df)

                # add to all_data
                if all_data != []:
                    # already have
                    all_data += df[1:]
                else:
                    all_data = df
                
                # add to provinces
                cur_provinces += [province]

        # add to province dic
        file_province[cur_file] = np.unique(cur_provinces).tolist()
    
    # print(file_province)

    # # print(np.array(all_data).shape)
    # print("\n".join([",".join(x) for x in all_data]))
    # # save to csv
    with open(out_path, "a+") as f:
        f.write("\n".join([",".join(x) for x in all_data])) 